In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import json
train_data = [json.loads(line) for line in open('Datasets/SST/data/train.jsonl')]
dev_data = [json.loads(line) for line in open('Datasets/SST/data/dev.jsonl')]

In [3]:
vec = TfidfVectorizer(min_df=2)
vec.fit([x['document'] for x in train_data])

TfidfVectorizer(min_df=2)

In [4]:
X_train, X_dev = vec.transform([x['document'] for x in train_data]), vec.transform([x['document'] for x in dev_data])
X_train.shape

(6920, 7134)

In [5]:
from sklearn.linear_model import LogisticRegressionCV

In [6]:
lr = LogisticRegressionCV()
lr.fit(X_train, [x['label'] for x in train_data])

/home/jainsarthak/miniconda3/envs/influence/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/jainsarthak/miniconda3/envs/influence/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

LogisticRegressionCV()

In [7]:
from sklearn.metrics import classification_report

In [9]:
print(classification_report([x['label'] for x in dev_data], lr.predict(X_dev)))

              precision    recall  f1-score   support

           0       0.79      0.77      0.78       428
           1       0.78      0.80      0.79       444

    accuracy                           0.79       872
   macro avg       0.79      0.79      0.79       872
weighted avg       0.79      0.79      0.79       872



In [10]:
import numpy as np

In [12]:
sim = np.multiply(X_dev, X_train.T)

In [40]:
l_train = np.array([x['label'] for x in train_data])
alpha = lr.C_[0] * (l_train - lr.predict_proba(X_train)[:, 1])

In [41]:
rp = 1. / (1 + np.exp(-sim @ alpha))

In [43]:
summ = np.concatenate([rp[:, None], lr.predict_proba(X_dev)[:, 1][:, None]], axis=-1)

In [45]:
np.corrcoef(summ[:, 0], summ[:, 1])

array([[1.       , 0.9998206],
       [0.9998206, 1.       ]])

In [56]:
inf_vals = np.multiply(sim.toarray(), alpha[None, :])

In [57]:
inf_vals.shape

(872, 6920)

In [58]:
import random

for i in range(5) :
    i = random.randint(0, 800)
    v = dev_data[i]
    print(v['document'])
    print("---")
    vals = inf_vals[i, :]
    top = np.argsort(vals)[-5:]
    for j in top[::-1] :
        print(vals[j], train_data[j]['document'])
    print("==="*10)

t his beguiling belgian fable very much its own droll and delicate little film has some touching things to say about what is important in life and why
---
0.2689957817110052 except it s much much better
0.2612592927184235 i am highly amused by the idea that we have come to a point in society where it has been deemed important enough to make a film in which someone has to be hired to portray richard dawson
0.24096692503493253 the best thing i can say about this film is that i ca n t wait to see what the director does next
0.20196055537616045 there is little question that this is a serious work by an important director who has something new to say about how in the flip flop of courtship we often reel in when we should be playing out
0.19096481498204915 while there s likely very little crossover appeal to those without much interest in the elizabethans as well as rank frustration from those in the know about rubbo s dumbed down tactics much ado about something is an amicable endeavor
it s